In [361]:
import numpy as np
import scqubits as scq

def tmon_model(EJ, EC):

    tmon = scq.Transmon(
        EJ=EJ,
        EC=EC,
        ng=0.0,
        ncut=31
    )
    
    tmon.eigenvals(evals_count=12)

    freq = np.diff(tmon.eigenvals(evals_count=12))[0]
    alpha = np.diff(np.diff(tmon.eigenvals(evals_count=12)))[0]

    return freq, alpha

def cost(x, *args):
    
    EJ = x[0]
    EC = x[1]
    
    freq_exp = args[0][0]
    alpha_exp = args[0][1]

    
    freq_model, alpha_model = tmon_model(EJ, EC)
    
    exp = np.array([freq_exp, alpha_exp])
    model = np.array([freq_model, alpha_model])
    
    error = np.sum( ((model-exp) / exp)**2 )
    
    return error

import scipy.optimize

def get_transmon_hamiltonian(freq, alpha):

    x0 = np.array([9.3, 0.195])

    args = [freq, alpha]

    result = scipy.optimize.minimize(cost, x0, tol=1e-8,bounds=[(0,100),(0,100)],args=args)

    result['x']

    tmon = scq.Transmon(
            EJ=9.44645349,
            EC=0.19360087,
            ng=0.0,
            ncut=31)
    
    return tmon.hamiltonian()

H = get_transmon_hamiltonian(3.62e9, 223e6)
w_q = np.diff(np.sort(np.linalg.eig(H)[0]))[0]  # bare qubit frequency
alpha = np.diff(np.diff(np.sort(np.linalg.eig(H)[0])))[0]  # anharmonicity
k6 = np.diff(np.diff(np.diff(np.sort(np.linalg.eig(H)[0]))))[0]  # sixth order term
print(w_q)
print(alpha)
print(k6)
np.diff(np.sort(np.linalg.eig(H)[0]))

3.6200069083915807
-0.2229990636909318
-0.026320487001292303


array([3.62000691e+00, 3.39700784e+00, 3.14768829e+00, 2.81267360e+00,
       2.67680627e+00, 1.29717497e+00, 3.95669791e+00, 7.61250236e-02,
       6.55369846e+00, 6.93010239e-04, 8.33045637e+00, 2.32219156e-06,
       9.95739000e+00, 3.69652753e-09, 1.15456881e+01, 3.18323146e-12,
       1.31169512e+01, 1.42108547e-13, 1.46795361e+01, 2.84217094e-13,
       1.62372848e+01, 0.00000000e+00, 1.77921547e+01, 3.69482223e-13,
       1.93452206e+01, 2.27373675e-13, 2.08971080e+01, 1.13686838e-13,
       2.24481987e+01, 5.68434189e-14, 2.39987348e+01, 1.70530257e-13,
       2.55488752e+01, 5.96855898e-13, 2.70987269e+01, 6.53699317e-13,
       2.86483642e+01, 2.27373675e-13, 3.01978394e+01, 1.70530257e-13,
       3.17471902e+01, 2.38742359e-12, 3.32964442e+01, 2.84217094e-12,
       3.48456222e+01, 1.76214598e-12, 3.63947395e+01, 5.68434189e-13,
       3.79438082e+01, 2.04636308e-12, 3.94928373e+01, 5.68434189e-13,
       4.10418341e+01, 9.09494702e-13, 4.25908043e+01, 4.54747351e-13,
      

Verify that the extracted transmon parameters give the same list of energies as the scqubits model.

In [343]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *


Nq = 10
w_q  = 2 * np.pi * 3.62e9 # bare qubit frequency
alpha = -223e6 * 2 * np.pi # anharmonicity
k6 = -26e6 * 2 * np.pi # sixth order term

q = destroy(Nq)

tlist = np.linspace(0,1e-7,10000)

H0 = w_q * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q)

np.diff(H0.eigenenergies()/(2e9*np.pi))


array([3.62 , 3.397, 3.148, 2.873, 2.572, 2.245, 1.892, 1.513, 1.108])

Create a dispersive readout Hamiltonian and use the eigenvectors to pick out which energies correspond to which cavity and transmon photon number. Then find the difference in energies between zero and one cavity photon, for all values of transmon photon number. This should give a state dependent $\chi$, keeping in mind that the transmon Hamiltonian is only up to sixth order. We could fit everything, but here I'm just playing with values to give close numbers.

The experimental data we have includes the dressed cavity frequency for four observed qubit states, and the dressed qubit transition frequency for the first and second transitions (equivalent to the anharmonicity).

From this we can get the fit bare cavity frequency.

The cavity frequencies are 5.72965, 5.72931, 5.72896 and 5.72867 GHz. The odd extra peaks are at (roughly) 5.7267 and 5.7305 GHz. 

The qubit frequencies are 3.620 GHz and 3.397 GHz. 

In [358]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

Nc = 10
Nq = 10

w_c  = 2 * np.pi * 5.7283e9 # bare resonator frequency
w_q  = 2 * np.pi * 3.623e9 # bare qubit frequency
alpha = -223e6 * 2 * np.pi # anharmonicity
g = 59.5e6 * 2 * np.pi
k6 = -26e6 * 2 * np.pi # sixth order term

c  = tensor(destroy(Nc), qeye(Nq))

q = tensor(qeye(Nc), destroy(Nq))

tlist = np.linspace(0,1e-7,10000)

H0 = w_c * c.dag() * c + w_q * q.dag() * q + (alpha/2)*q.dag()*q.dag()*q*q  + (k6/6)*(q.dag()*q.dag()*q.dag()*q*q*q) - g*(c.dag() - c)*(q.dag() - q)

H = np.array(H0)

cav_0 = np.zeros(Nq)
cav_1 = np.zeros(Nq)

for i in range(0,Nc*Nq):

    eigval = H0.eigenenergies()[i]
    eigvect = np.abs(np.reshape(np.array(H0.eigenstates()[1][i]),(Nc,Nq)))
    cav_is, qb_is = np.where(eigvect==np.max(eigvect))
    cav_i = cav_is[0]
    qb_i = qb_is[0]
    
    if cav_i == 0:
        cav_0[qb_i] = eigval
        
    elif cav_i == 1:
        cav_1[qb_i] = eigval
      

print('Chi from the model: ' + str((cav_1-cav_0)/(2e9*np.pi)))
print('Dressed qb freq: ' + str((cav_0[1]-cav_0[0])/(2e9*np.pi)))
print('Dressed cav freq: ' + str((cav_1[0]-cav_0[0])/(2e9*np.pi)))

Chi from the model: [5.72960186 5.72926219 5.72895972 5.72869485 5.72846357 5.72825947
 5.72807485 5.72790139 5.72772751 5.72605769]
Dressed qb freq: 3.620922288841517
Dressed cav freq: 5.729601864176648


This suggests the bare cav frequency is about 5.7283 GHz, which does not correspond to the 5.7267 GHz line.